In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision
from core.models import ResNetEncoder
encoder = ResNetEncoder(out_features=1024)

/home/heon/anaconda3/envs/camelyon/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/heon/anaconda3/envs/camelyon/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
import os
import glob

from torchvision.transforms import RandomCrop, GaussianBlur, RandomEqualize, RandomRotation, RandomAffine, Compose, ToTensor
from core.datasets import SimCLRDataSet
from matplotlib import pyplot as plt

images = glob.glob(os.path.join("/home/heon/heon_vast/camelyon17_patches_new/RUMC/malignant", "*"))
dataset = SimCLRDataSet(
    images, 
    augmentations=[
        GaussianBlur((3,3)), 
        RandomEqualize(), 
        RandomRotation(degrees=(0, 360)), 
        RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75))
    ],
    transforms=Compose([ToTensor()]),
    device="cuda:1"
)

In [4]:
from core.trainer import SimCLRTrainer
from core.loss import SimCLRLoss
dataloder = torch.utils.data.DataLoader(dataset, batch_size=16)
criterion = SimCLRLoss()
trainer = SimCLRTrainer(encoder, criterion)

In [5]:
encoder.to("cuda:1")
a, b = trainer.run_epoch(dataloder, phase="train", epoch=1)

In [18]:
from core.loss import SimCLRLoss
loss = SimCLRLoss(device="cuda:1")
loss(a,b)

tensor(3.4497, device='cuda:1', grad_fn=<NllLossBackward0>)